#### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [10]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint

In [11]:
url = 'https://spb.hh.ru'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)' 
           'Chrome/96.0.4664.93 Safari/537.36'
          }
page = 0
last_page = 1
vacancy_text = 'analyst'
params = {
    'text': vacancy_text,
    'page': page
}

In [12]:
vacancies_info = []
page = 0
while page < last_page:
    
        response = requests.get(url+'/vacancies/analyst',params=params,headers=headers)
        
        if response.ok:
            soup = bs(response.text, 'html.parser')
            vacancies = soup.find_all('div', {'class': ['vacancy-serp-item-body__main-info']})
    
            for vacancy in vacancies:
                vacancy_info = {}

                info = vacancy.find('a', {'data-qa': 'serp-item__title'})
                name = info.text
                
                link = info.get('href')
                min_salary = None
                max_salary = None
                currency = None

                try:
                    salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
                    data = salary.replace('\u202f', '').split()

                    if data[0] == 'от':
                        min_salary = int(data[1])
                        max_salary = None

                    elif data[0] == 'до':
                        min_salary = None
                        max_salary = int(data[1])

                    else:
                        min_salary = int(data[0])
                        max_salary = int(data[2])
                    currency = data[-1]

                except:
                    min_price = None
                    max_price = None
                    currency = None

                vacancy_info['source'] = url
                vacancy_info['name'] = name
                vacancy_info['link'] = link
                vacancy_info['min_price'] = min_salary
                vacancy_info['max_price'] = max_salary
                vacancy_info['currency'] = currency

                vacancies_info.append(vacancy_info)

            page += 1

            count = soup.find_all('a', {'class': 'bloko-button'})
            last_page = int(count[-2].text)

        else:
            break

In [ ]:
# pprint(vacancies_info)

In [13]:
len(vacancies_info)

2032

In [14]:
conda install pymongo

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done


In [15]:
from pymongo import MongoClient

In [16]:
client = MongoClient('127.0.0.1', 27017)

In [32]:
db = client['vacancies_hh']

In [33]:
collection_vacancies_hh = db.collection_vacancies_hh

In [34]:
def check_vacancies(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh.find({'link': vacancy['link']}))):
      collection_vacancies_hh.insert_one(vacancy)

In [35]:
check_vacancies(vacancies_info)

In [36]:
vacancies_info_new = list(collection_vacancies_hh.find())

In [37]:
len(vacancies_info_new)

59